<a href="https://colab.research.google.com/github/YosyValero/jvalero59_TFG/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline TFG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [220]:
''' 16/06 = NO VALIDO ==> ruta = 'https://drive.google.com/drive/folders/1QO-S6eSqUXNwZt-oU6G5H1DwmN-vSbnC?usp=sharing'
path_tsv = ruta + '/tsv/'
'''

In [223]:
import numpy as np
import pandas as pd

In [224]:
path_tsv = '/content/drive/MyDrive/Dataset/tsv/'

In [225]:
df_dissea =  pd.read_csv(path_tsv + 'dissea.tsv', sep='\t', header=0).loc[:, ('tweets_id', 'type')].rename(columns={'tweets_id': 'tweet_id'})

In [ ]:
df_prof =  pd.read_csv(path_tsv + 'prof.tsv', sep='\t', header=0).loc[:, ('tweet_id', 'type')]

In [ ]:
df_prof.drop(df_prof[df_prof['type']=='-'].index, inplace=True)

In [ ]:
df_prof = df_prof.replace(['PROFESION', 'SITUACION_LABORAL', 'ACTIVIDAD', 'FIGURATIVA'], ['NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD'])

In [ ]:
df_prof_norep = df_prof.drop_duplicates(ignore_index=True)

In [ ]:
df_disse_norep = df_dissea.drop_duplicates(ignore_index=True)

In [ ]:
muestras_min = min([len(df_disse_norep), len(df_prof_norep)])

In [ ]:
muestras = 600

In [ ]:
if muestras_min < muestras:
  muestras = muestras_min

In [ ]:
df_xamples_diss = df_disse_norep.head(muestras)

In [ ]:
df_xamples_nodiss = df_prof_norep.head(muestras)

In [ ]:
df_xamples_nodiss = df_xamples_nodiss.rename(index=lambda x: x + muestras)

In [ ]:
df_tot = pd.concat([df_xamples_diss, df_xamples_nodiss])


In [ ]:
df_tot.insert(2, 'content_tuit', value='')

In [ ]:
def get_tuit(type_tuit, name_tuit):
  sub_path = path_tsv + 'train-valid-txt-files/'
  if type_tuit == 'ENFERMEDAD':
    path_tuit = sub_path + 'dissea'
  else:
    path_tuit = sub_path + 'prof'

  path_tuit = path_tuit + '/training/' + name_tuit + '.txt'
  tuit = open(path_tuit, mode='r', encoding='utf-8')
  content_tuit = tuit.read()
  tuit.close()

  return content_tuit

In [ ]:
for i in range(len(df_tot)):
    name_tuit = df_tot.loc[i, 'tweet_id']
    type_tuit = df_tot.loc[i, 'type']
    df_tot.at[i, 'content_tuit'] = get_tuit(type_tuit, str(name_tuit))

In [ ]:
df_tot